## Import

In [1]:
# notebooky stuff
%load_ext autoreload
%autoreload 2
from IPython.display import display

import sys 
sys.path.append('../../src/modules')

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import eumf_data, eumf_eval, eumf_pipeline
from sklearn import preprocessing, linear_model, model_selection, ensemble, feature_selection
import seaborn as sns
import itertools

# pandas pretty output
pd.set_option('display.min_rows', 20)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)

## Data

### Load all data

In [2]:
# migration rates
df_values = eumf_data.load_registrations_from_csv(impute_missing=True)

# google trends
df_trends = eumf_data.load_trends_from_csv()

countries = eumf_data.get_countries()
keyword_ids = df_trends.columns.levels[0].tolist()

df_gdp = eumf_data.read_gdp()
df_unempl = eumf_data.read_unempl()


### Join data, combine countries

In [3]:
country_combinations = [
    ["GR", "CY"],
    ["LV", "LT", "EE"],
    ["BE", "NL", "LU"],
    ["CZ", "SK"],
    ["SE", "FI", "DK"],
    ["AT", "CH"]
]

panel = df_values.join(df_trends, how="outer")
panel_3m = panel.resample("3M", closed="left").mean()

panel_comb = eumf_data.combine_countries(panel, combinations=country_combinations)
panel_comb_3m = eumf_data.combine_countries(panel_3m, combinations=country_combinations)
df_gdp_comb = eumf_data.combine_countries(df_gdp, combinations=country_combinations)
df_unempl_comb = eumf_data.combine_countries(
    df_unempl, combinations=country_combinations, average=True
)
# note: strictly, unweighted average is wrong for unemployment, but should work in most cases

panel_comb_3m_macro = panel_comb_3m.join(df_gdp_comb).join(df_unempl_comb)

panel_comb_3m_macro["2017":"2018"]


10                                                                                                                        11                                                                                                                                            112                                                                                                               ...   value                                                                                                                                     gdp                                                                                                                                                 unempl                                                                                                       
country     AT+CH  BE+NL+LU  BG  CZ+SK    ES      FR      GB   GR+CY  HR   HU   IE     IT   LV+LT+EE    PL    PT   RO  SE+FI+DK  SI   AT+CH  BE+NL+LU    BG    CZ+SK     ES      FR      GB    GR+CY    HR      HU      IE      IT   LV+LT+EE    PL      PT      RO   SE+FI+DK  SI   AT+CH  BE+NL+LU    BG    CZ+SK      ES      FR      GB    GR+CY     HR      HU      IE      IT   LV+LT+EE    PL    ...     ES        FR        GB      GR+CY       HR        HU       IE        IT     LV+LT+EE     PL        PT        RO     SE+FI+DK    SI     AT+CH   BE+NL+LU    BG    CZ+SK     ES      FR      GB    GR+CY      HR      HU      IE       IT   LV+LT+EE    PL      PT      RO   SE+FI+DK    SI   AT+CH  BE+NL+LU  BG  CZ+SK   ES   FR   GB  GR+CY    HR   HU   IE    IT  LV+LT+EE  PL   PT   RO  SE+FI+DK  SI 
date                                                                                                                                                                                                                                                                                                                                                                                                    ...                                                                                                                                                                                                                                                                                                                                                                                                              
2017-03-31  10.524  16.048   0.0   0.0  26.667  21.857  21.381   0.0  0.0  0.0  0.0  14.095    0.0     6.095  0.0  0.0  21.286   0.0  34.714  60.190    2.190  10.857  38.714  29.810  53.381  12.810  0.000   0.619  11.095  22.048    0.0    11.952  12.714  11.571  26.524   0.0  97.190  178.952  42.238  127.048  33.143  63.333  47.286  29.619  63.095  62.810  59.333  14.905  179.429  39.524  ...  1689.000  1268.333  1080.333  2399.000  4459.000  3747.667  156.000  5255.333  1396.333  12264.667  866.333  18186.333  654.333  420.333  28780.0  42970.0  1510.0  7620.0  5950.0  8450.0  9080.0   9270.0  2590.0  2820.0  14640.0  6800.0  10490.0  2760.0  4520.0  1850.0  33780.0  4790.0  2.85    6.433   6.6  6.00  18.2  9.6  4.6  17.35  12.7  4.3  7.4  11.6   7.333   5.3  9.8  5.3   7.167   7.4
2017-06-30   4.952  14.810   0.0   0.0  21.048  13.952  18.857   0.0  0.0  0.0  0.0  15.286    0.0     6.524  0.0  0.0  27.714   0.0  39.286  61.619   10.619   7.714  24.381  27.952  55.714   8.286  2.333   7.095  20.619  17.286    0.0    13.810  23.143  11.143  32.857   0.0  85.619  168.905  35.190   83.429  23.810  57.857  48.000  36.476  33.048  27.905  62.810  14.286  126.667  21.476  ...  1427.667  1023.333  1049.000  2163.000  4121.333  3637.667  164.667  4699.000  1407.000  12940.000  609.667  19264.000  520.333  365.000  29250.0  44540.0  1800.0  8470.0  6330.0  8560.0  8980.0  10000.0  2990.0  3200.0  14700.0  7140.0  11780.0  2960.0  4790.0  2200.0  35590.0  5260.0  2.70    5.867   6.2  5.75  17.3  9.6  4.4  16.70  11.5  4.3  6.7  11.3   7.667   5.1  9.2  4.9   7.133   6.6
2017-09-30   4.905  16.095   0.0   0.0  12.4

## Experiments

In [10]:
T_TEST_MIN = "2014"
T_TEST_MAX = "2014"



### Which AR order?

up to 8

In [12]:
### TRAINING
t_min = "2010"
t_max = "2019"

n_years_eff = int(t_max) - int(t_min) - 1

cv_default = model_selection.KFold(n_splits=n_years_eff, shuffle=False)

feature_combinations = [["value"], ["value", "19"], ["19"]]

lag_order = [1, 2, 3, 4, 5, 6, 7, 8]

tuners = []
cv_scores, test_scores = [], []
train_stackeds, test_stackeds, train_unstackeds = [], [], []
model_names = []

# params = {}
params = {
    "randomforestregressor__max_features": ["auto", "sqrt"],
    "randomforestregressor__min_samples_leaf": [1, 2, 4, 8],
    "randomforestregressor__min_samples_split": [2, 4, 8, 16],
}

for features, order in itertools.product(feature_combinations, lag_order):

    model_names.append("+".join(features) + "_" + str(order))

    labeled = eumf_pipeline.prepare_data(
        panel_comb_3m_macro,
        columns=features,
        lags=list(range(1, order + 1)),
        t_min=t_min,
        t_max=t_max,
    )
    transformed = eumf_pipeline.transform_data(labeled)
    train, test = eumf_pipeline.split_data(
        transformed, t_test_min=T_TEST_MIN, t_test_max=T_TEST_MAX
    )
    train_stacked, test_stacked = eumf_pipeline.stack_data(train, test)

    train_stackeds.append(train_stacked)
    train_unstackeds.append(train)
    test_stackeds.append(test_stacked)

    tuner = eumf_pipeline.train_reg_model(
        train_stacked,
        reg=ensemble.RandomForestRegressor(random_state=42),
        extra_pipeline_steps=[preprocessing.StandardScaler()],
        params=params,
        scoring=eumf_eval.scorer_rmse,
    )
    tuners.append(tuner)

    cv_score = eumf_eval.score_cv(tuner.best_estimator_, train_stacked, cv=cv_default,)
    cv_scores.append(cv_score)

    test_score = eumf_eval.score_test(tuner.best_estimator_, test_stacked,)
    test_scores.append(test_score)


In [13]:
pd.DataFrame({k: t.best_params_ for k, t in zip(model_names, tuners)}).transpose()

,randomforestregressor__max_features,randomforestregressor__min_samples_leaf,randomforestregressor__min_samples_split
value_1,auto,8,2
value_2,auto,8,2
value_3,auto,8,2
value_4,auto,8,2
value_5,auto,4,2
value_6,auto,4,2
value_7,auto,4,2
value_8,auto,4,2
value+19_1,auto,4,16
value+19_2,auto,8,2


In [14]:
eumf_eval.agg_multiple_cv_scores(cv_scores, model_names, use_quantiles=True)


fit_time               score_time               test_mae               test_rmse               test_explained_variance               test_r2_mod              
             0.25    0.50   0.75     0.25     0.50   0.75    0.25    0.50   0.75     0.25    0.50   0.75            0.25           0.50   0.75      0.25     0.50   0.75 
value_1      0.116   0.122  0.129    0.010    0.011  0.013  -0.076  -0.067 -0.060   -0.132  -0.091 -0.082           0.228          0.376  0.536     0.243    0.454  0.656
value_2      0.126   0.129  0.131    0.011    0.012  0.013  -0.078  -0.069 -0.059   -0.132  -0.094 -0.080           0.266          0.354  0.513     0.277    0.424  0.637
value_3      0.137   0.138  0.140    0.010    0.011  0.012  -0.079  -0.070 -0.059   -0.133  -0.096 -0.081           0.270          0.348  0.506     0.282    0.400  0.630
value_4      0.152   0.162  0.174    0.012    0.012  0.016  -0.078  -0.069 -0.057   -0.131  -0.094 -0.078           0.324          0.375  0.516     0.336    0.425  0.630
value_5      0.185   0.193  0.201    0.012    0.013  0.016  -0.077  -0.066 -0.058   -0.122  -0.093 -0.079           0.309          0.358  0.532     0.323    0.413  0.659
value_6      0.222   0.237  0.259    0.013    0.014  0.015  -0.077  -0.066 -0.059   -0.122  -0.093 -0.078           0.318          0.362  0.522     0.327    0.415  0.652
value_7      0.204   0.207  0.217    0.011    0.012  0.013  -0.076  -0.066 -0.058   -0.122  -0.093 -0.078           0.320          0.366  0.530     0.328    0.419  0.654
value_8      0.205   0.218  0.239    0.010    0.011  0.013  -0.077  -0.066 -0.057   -0.122  -0.094 -0.078           0.322          0.385  0.533     0.329    0.436  0.658
value+19_1   0.127   0.133  0.138    0.011    0.011  0.011  -0.074  -0.069 -0.059   -0.114  -0.092 -0.082           0.228          0.358  0.542     0.239    0.447  0.660
value+19_2   0.151   0.153  0.174    0.011    0.011  0.012  -0.075  -0.067 -0.057   -0.125  -0.090 -0.077           0.309          0.378  0.511     0.335    0.436  0.646
value+19_3   0.175   0.181  0.190    0.011    0.012  0.012  -0.077  -0.068 -0.056   -0.127  -0.090 -0.076           0.316          0.387  0.499     0.343    0.435  0.643
value+19_4   0.227   0.243  0.257    0.011    0.011  0.013  -0.076  -0.067 -0.058   -0.119  -0.088 -0.079           0.310          0.378  0.513     0.315    0.444  0.648
value+19_5   0.261   0.297  0.335    0.012    0.014  0.015  -0.074  -0.065 -0.057   -0.120  -0.087 -0.077           0.331          0.405  0.519     0.334    0.474  0.659
value+19_6   0.331   0.371  0.414    0.013    0.016  0.017  -0.073  -0.064 -0.056   -0.120  -0.087 -0.077           0.338          0.407  0.520     0.347    0.478  0.658
value+19_7   0.318   0.325  0.329    0.012    0.012  0.013  -0.072  -0.065 -0.057   -0.118  -0.087 -0.076           0.335          0.408  0.532     0.345    0.476  0.668
value+19_8   0.338   0.355  0.366    0.012    0.012  0.013  -0.073  -0.065 -0.056   -0.116  -0.087 -0.076           0.336          0.411  0.509     0.346    0.479  0.655
19_1         0.106   0.114  0.121    0.012    0.013  0.014  -0.116  -0.100 -0.080   -0.170  -0.134 -0.110          -0.226          0.044  0.221    -0.469   -0.054  0.326
19_2         0.124   0.130  0.135    0.013    0.016  0.016  -0.110  -0.091 -0.071   -0.161  -0.125 -0.102          -0.070          0.058  0.220    -0.283    0.032  0.414
19_3         0.121   0.122  0.129    0.012    0.013  0.013  -0.112  -0.090 -0.067   -0.165  -0.125 -0.098          -0.140          0.028  0.215    -0.297    0.081  0.508
19_4         0.145   0.170  0.187    0.016    0.017  0.018  -0.110  -0.086 -0.067   -0.166  -0.120 -0.099          -0.178          0.098  0.163    -0.187    0.065  0.537
19_5         0.176   0.183  0.205    0.012    0.013  0.015  -0.102  -0.084 -0.071   -0.157  -0.119 -0.098          -0.165          0.010  0.270    -0.176    0.172  0.495
19_6         0.144   0.158  0.161    0.014    0.016  0.017  -0.102  -0.080 -0.064   -0.165  -0.112 -0

up to 4, one more year in training

In [30]:
### TRAINING
t_min = "2009"
t_max = "2019"

n_years_eff = int(t_max) - int(t_min) - 1

cv_default = model_selection.KFold(n_splits=n_years_eff, shuffle=False)

feature_combinations = [["value"], ["value", "19"], ["19"]]

lag_order = [1, 2, 3, 4]

tuners = []
cv_scores, test_scores = [], []
train_stackeds, test_stackeds, train_unstackeds = [], [], []
model_names = []

# params = {}
params = {
    "randomforestregressor__max_features": ["auto", "sqrt"],
    "randomforestregressor__min_samples_leaf": [1, 2, 4, 8],
    "randomforestregressor__min_samples_split": [2, 4, 8, 16],
}

for features, order in itertools.product(feature_combinations, lag_order):

    model_names.append("+".join(features) + "_" + str(order))

    labeled = eumf_pipeline.prepare_data(
        panel_comb_3m_macro,
        columns=features,
        lags=list(range(1, order + 1)),
        t_min=t_min,
        t_max=t_max,
    )
    transformed = eumf_pipeline.transform_data(labeled)
    train, test = eumf_pipeline.split_data(
        transformed, t_test_min=T_TEST_MIN, t_test_max=T_TEST_MAX
    )
    train_stacked, test_stacked = eumf_pipeline.stack_data(train, test)

    train_stackeds.append(train_stacked)
    train_unstackeds.append(train)
    test_stackeds.append(test_stacked)

    tuner = eumf_pipeline.train_reg_model(
        train_stacked,
        reg=ensemble.RandomForestRegressor(random_state=42),
        extra_pipeline_steps=[preprocessing.StandardScaler()],
        params=params,
        scoring=eumf_eval.scorer_rmse,
    )
    tuners.append(tuner)

    cv_score = eumf_eval.score_cv(tuner.best_estimator_, train_stacked, cv=cv_default,)
    cv_scores.append(cv_score)

    test_score = eumf_eval.score_test(tuner.best_estimator_, test_stacked,)
    test_scores.append(test_score)


In [31]:
pd.DataFrame({k: t.best_params_ for k, t in zip(model_names, tuners)}).transpose()

,randomforestregressor__max_features,randomforestregressor__min_samples_leaf,randomforestregressor__min_samples_split
value_1,auto,4,16
value_2,auto,4,16
value_3,auto,4,16
value_4,auto,4,16
value+19_1,auto,2,16
value+19_2,auto,2,16
value+19_3,auto,4,16
value+19_4,auto,4,16
19_1,sqrt,8,2
19_2,sqrt,4,16


In [32]:
eumf_eval.agg_multiple_cv_scores([df[1:] for df in cv_scores], model_names, use_quantiles=True)


fit_time               score_time               test_mae               test_rmse               test_explained_variance               test_r2_mod              
             0.25    0.50   0.75     0.25     0.50   0.75    0.25    0.50   0.75     0.25    0.50   0.75            0.25           0.50   0.75      0.25     0.50   0.75 
value_1      0.178   0.189  0.200    0.016    0.016  0.018  -0.075  -0.068 -0.062   -0.121  -0.094 -0.085           0.196          0.367  0.544     0.198    0.436  0.672
value_2      0.161   0.162  0.174    0.012    0.013  0.013  -0.074  -0.071 -0.060   -0.117  -0.097 -0.081           0.248          0.355  0.531     0.244    0.423  0.665
value_3      0.173   0.175  0.178    0.013    0.013  0.013  -0.077  -0.071 -0.060   -0.116  -0.098 -0.082           0.232          0.335  0.524     0.248    0.381  0.660
value_4      0.183   0.187  0.190    0.012    0.013  0.013  -0.075  -0.067 -0.056   -0.113  -0.096 -0.079           0.296          0.355  0.539     0.288    0.411  0.666
value+19_1   0.167   0.167  0.177    0.012    0.013  0.013  -0.073  -0.068 -0.061   -0.112  -0.094 -0.084           0.156          0.355  0.547     0.173    0.442  0.664
value+19_2   0.196   0.197  0.209    0.012    0.013  0.013  -0.072  -0.068 -0.061   -0.111  -0.095 -0.084           0.210          0.356  0.551     0.206    0.427  0.667
value+19_3   0.216   0.220  0.228    0.013    0.013  0.014  -0.074  -0.071 -0.058   -0.112  -0.093 -0.079           0.304          0.370  0.537     0.302    0.438  0.661
value+19_4   0.289   0.308  0.366    0.014    0.016  0.018  -0.074  -0.068 -0.058   -0.112  -0.089 -0.078           0.318          0.393  0.545     0.320    0.459  0.664
19_1         0.115   0.118  0.120    0.013    0.014  0.014  -0.119  -0.103 -0.083   -0.182  -0.136 -0.113          -0.220          0.010  0.230    -0.539   -0.142  0.317
19_2         0.117   0.121  0.125    0.012    0.013  0.013  -0.113  -0.094 -0.074   -0.166  -0.128 -0.105          -0.099          0.016  0.256    -0.371   -0.029  0.400
19_3         0.161   0.170  0.175    0.016    0.017  0.019  -0.111  -0.092 -0.070   -0.162  -0.128 -0.099          -0.151          0.026  0.275    -0.345    0.050  0.496
19_4         0.115   0.118  0.124    0.011    0.012  0.012  -0.110  -0.087 -0.066   -0.169  -0.122 -0.100          -0.150          0.061  0.203    -0.246    0.054  0.513

### Add absolute values to GT

In [33]:
### TRAINING

t_min = "2010"
t_max = "2019"
n_years_eff = int(t_max) - int(t_min) - 1
cv_default = model_selection.KFold(n_splits=n_years_eff, shuffle=False)

feature_combinations = [["value", "19"], ["19"], ["19", "gdp", "unempl"]]
w_absolutes = [False, True]

tuners = []
cv_scores, test_scores = [], []
train_stackeds, test_stackeds, train_unstackeds = [], [], []
model_names = []

# params = {}
params = {
    "randomforestregressor__max_features": ["auto", "sqrt"],
    "randomforestregressor__min_samples_leaf": [1, 2, 4, 8],
    "randomforestregressor__min_samples_split": [2, 4, 8, 16],
}

for features, w_absolutes in itertools.product(feature_combinations, w_absolutes):

    model_names.append("+".join(features) + "_" + str(w_absolutes))

    labeled = eumf_pipeline.prepare_data(
        panel_comb_3m_macro, columns=features, lags=list(range(1, 6)), t_min="2010"
    )
    transformed = eumf_pipeline.transform_data(labeled)
    if w_absolutes:
        extra_cols = [c for c in labeled.x.columns if c[0].startswith("19")]
        transformed.x = transformed.x.join(labeled.x[extra_cols], rsuffix="_abs")
    train, test = eumf_pipeline.split_data(
        transformed, t_test_min=T_TEST_MIN, t_test_max=T_TEST_MAX
    )
    train_stacked, test_stacked = eumf_pipeline.stack_data(train, test)

    train_stackeds.append(train_stacked)
    train_unstackeds.append(train)
    test_stackeds.append(test_stacked)

    tuner = eumf_pipeline.train_reg_model(
        train_stacked,
        reg=ensemble.RandomForestRegressor(random_state=42),
        extra_pipeline_steps=[preprocessing.StandardScaler()],
        params=params,
        scoring=eumf_eval.scorer_rmse,
    )
    tuners.append(tuner)

    cv_score = eumf_eval.score_cv(tuner.best_estimator_, train_stacked, cv=cv_default,)
    cv_scores.append(cv_score)

    test_score = eumf_eval.score_test(tuner.best_estimator_, test_stacked,)
    test_scores.append(test_score)


In [34]:
pd.DataFrame({k: t.best_params_ for k, t in zip(model_names, tuners)}).transpose()

,randomforestregressor__max_features,randomforestregressor__min_samples_leaf,randomforestregressor__min_samples_split
value+19_False,auto,4,2
value+19_True,auto,8,2
19_False,auto,8,2
19_True,sqrt,1,4
19+gdp+unempl_False,sqrt,1,16
19+gdp+unempl_True,sqrt,1,8


In [35]:
eumf_eval.agg_multiple_cv_scores(cv_scores, model_names, use_quantiles=True)


fit_time               score_time               test_mae               test_rmse               test_explained_variance               test_r2_mod              
                      0.25    0.50   0.75     0.25     0.50   0.75    0.25    0.50   0.75     0.25    0.50   0.75            0.25           0.50   0.75      0.25     0.50   0.75 
value+19_False        0.257   0.283  0.308    0.013    0.013  0.014  -0.074  -0.065 -0.057   -0.120  -0.087 -0.077           0.331          0.405  0.519     0.334    0.474  0.659
value+19_True         0.266   0.286  0.292    0.011    0.012  0.013  -0.074  -0.066 -0.054   -0.124  -0.088 -0.073           0.347          0.435  0.501     0.375    0.486  0.652
19_False              0.194   0.205  0.234    0.013    0.014  0.016  -0.102  -0.084 -0.071   -0.157  -0.119 -0.098          -0.165          0.010  0.270    -0.176    0.172  0.495
19_True               0.147   0.151  0.155    0.012    0.012  0.013  -0.097  -0.084 -0.064   -0.154  -0.117 -0.093          -0.072          0.136  0.292    -0.144    0.204  0.539
19+gdp+unempl_False   0.211   0.245  0.256    0.018    0.020  0.025  -0.109  -0.075 -0.062   -0.150  -0.101 -0.086           0.127          0.181  0.261     0.151    0.277  0.409
19+gdp+unempl_True    0.235   0.242  0.265    0.016    0.022  0.028  -0.107  -0.077 -0.059   -0.147  -0.105 -0.084           0.110          0.137  0.246     0.117    0.254  0.469